# TP #1 - Estimation des courbures locales sur des maillages 3D

On vous propose dans ce TP d'évaluer et d'afficher les courbures locales d'un maillage 3D en déterminant sur chaque sommet de ce même maillage les valeurs propres associées à sa la matrice de covariance.

Vous vous référerez pour cela à section 2 de l'article suivant :

M. Pauly, M. Gross and L. P. Kobbelt, **Efficient simplification of point-sampled surfaces**, *IEEE Visualization, 2002. VIS 2002., Boston, MA, USA, 2002, pp. 163-170, doi: 10.1109/VISUAL.2002.1183771.*

<img src="p_Pau021.jpg" width="250" />

Le fichier Pdf de cet article est disponible <a href="p_Pau021.pdf">ici</a>

**Remarque** : pour le calcul des valeurs propres référerez-vous à <a href="https://numpy.org/doc/stable/reference/routines.linalg.html">NumPy</a>. 

In [7]:
from pygel3d import hmesh, jupyter_display as jd
m = hmesh.load("bunny.obj")

C'est à vous de jouer maintenant !

In [8]:
def show_cov(i_start,i_end, max_print=None):
    global cov_matrix_l
    global eigenvalues_l
    if max_print is None:
        for i in range(i_start,i_end):
            eigenvalues = eigenvalues_l[i]
            print(f"Eigenvalues at vertex {i}: {eigenvalues}")
    else:
        for i in range(i_start,i_start+(max_print//2)):
            eigenvalues = eigenvalues_l[i]
            print(f"Eigenvalues at vertex {i}: {eigenvalues}")
        print("...")
        for i in range(i_end-(max_print//2),i_end):
            eigenvalues = eigenvalues_l[i]
            print(f"Eigenvalues at vertex {i}: {eigenvalues}")

In [13]:
show_cov(int(0),len(eigenvalues_l),int(10))

Eigenvalues at vertex 0: [1.57166553e-03 1.74907220e-05 6.67009383e-04]
Eigenvalues at vertex 1: [2.00613902e-03 8.51306455e-05 5.73834264e-04]
Eigenvalues at vertex 2: [3.21359777e-03 8.38858408e-04 2.07945070e-06]
Eigenvalues at vertex 3: [4.25474359e-03 8.72792862e-04 2.94122277e-05]
Eigenvalues at vertex 4: [5.33940839e-03 6.75536291e-04 2.38631823e-05]
...
Eigenvalues at vertex 2498: [0.00267365 0.00091845 0.00012023]
Eigenvalues at vertex 2499: [7.79853348e-03 3.19750580e-03 1.42002197e-06]
Eigenvalues at vertex 2500: [3.17455678e-03 2.41867130e-03 3.57902185e-06]
Eigenvalues at vertex 2501: [5.16351206e-03 5.08919895e-04 1.99325107e-05]
Eigenvalues at vertex 2502: [2.84551641e-05 1.35539263e-03 2.71792091e-03]


```text
Second version
```

In [66]:
from pygel3d import hmesh, jupyter_display as jd
import numpy as np

def compute_covariance_matrix(neighbors, center):
    """Compute the covariance matrix from neighbors and the center."""
    centered_vectors = [p - center for p in neighbors]
    covariance_matrix = np.cov(np.array(centered_vectors).T)
    return covariance_matrix

def compute_eigenvalues(cov_matrix):
    """Compute the eigenvalues of a covariance matrix."""
    eigenvalues, _ = np.linalg.eig(cov_matrix)
    return eigenvalues

def build_vertex_adjacency_map(mesh):
    """Build a map of vertex indices to their neighboring vertex indices."""
    adjacency_map = {i: set() for i in range(len(mesh.positions()))}  # Initialize adjacency map
    for face in mesh.faces():
        vertices = mesh.circulate_face(face)  # Retrieve vertices of the face
        for v in vertices:
            adjacency_map[v].update(vertices)  # Add all face vertices to the adjacency map
            adjacency_map[v].discard(v)  # Remove the vertex itself
    return adjacency_map

def get_neighbors(mesh, pos, index, levels, adjacency_map):
    """Retrieve neighbors up to a specified level for a given vertex index."""
    neighbors = set([index])
    for _ in range(levels):
        current_level = set()
        for v in neighbors:
            current_level.update(adjacency_map[v])
        neighbors.update(current_level)
    neighbors.discard(index)  # Remove the center index
    return [pos[i] for i in neighbors]

def compute_eigenvalues_for_mesh(mesh, levels):
    """Compute eigenvalues for all vertices in the mesh considering the specified levels of neighbors."""
    pos = mesh.positions()
    adjacency_map = build_vertex_adjacency_map(mesh)  # Build adjacency map once
    eigenvalues_list = []
    
    for i, center in enumerate(pos):
        neighbors = get_neighbors(mesh, pos, i, levels, adjacency_map)
        if neighbors:
            cov_matrix = compute_covariance_matrix(neighbors, center)
            eigenvalues = compute_eigenvalues(cov_matrix)
            eigenvalues_list.append(eigenvalues)
        else:
            eigenvalues_list.append(None)  # Handle vertices with no neighbors

    return eigenvalues_list

def visualize_eigenvalues(mesh, eigenvalues_list, eigenvalue_index=0):
    """Visualize a specific eigenvalue (first value from each array) as a colormap on the mesh."""
    
    # Get the vertex positions
    vertex_positions = mesh.positions()

    # Prepare an empty list to store the intensity
    colors = []

    # Extract the first eigenvalue from each eigenvalue array in eigenvalues_list
    eigenvalues = [eigenvalues[0] if eigenvalues is not None else 0.0 for eigenvalues in eigenvalues_list]

    # Normalize the eigenvalues to the range [0, 1]
    min_val, max_val = min(eigenvalues), max(eigenvalues)
    normalized_eigenvalues = [(eigenvalue - min_val) / (max_val - min_val) if max_val > min_val else 0.5 for eigenvalue in eigenvalues]

    for i, (x, y, z) in enumerate(vertex_positions):
        # Get the normalized color corresponding to the eigenvalue at this vertex
        color_value = normalized_eigenvalues[i] if i < len(normalized_eigenvalues) else 0.5  # Default to 0.5 if out of range

        # Append the intensity for the vertices for colors
        colors.append(color_value)

    # Convert the list to a numpy array
    colors = np.array(colors)
    
    return colors

def visualize_eigenvalues(mesh, eigenvalues_list):
    # Normalize the 1x3 matrix
    intensity = np.array([np.linalg.norm(v) for v in eigenvalues_list])
    # Normalize value between 0 and 1
    min_val, max_val = np.min(intensity), np.max(intensity)
    normalized_intensity = (intensity - min_val) / (max_val - min_val)
    
    return normalized_intensity

# Example usage
mesh = hmesh.load("bunny.obj")
levels = 8 # Specify the level of neighbors to consider
eigenvalues_list = compute_eigenvalues_for_mesh(mesh, levels)
colors = visualize_eigenvalues(mesh, eigenvalues_list)
print(colors)
# Visualize the mesh with the color data (where colors are in the format (r, g, b, a))
jd.display(mesh, data=colors)

[0.36948155 0.32471831 0.54968853 ... 0.61947635 0.51290482 0.32193683]


FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([ 562,    1, 1288, ...,  189, 1159, 1459]),
              'intensity': array([0.36948155, 0.32471831, 0.54968853, ..., 0.61947635, 0.51290482,
                                  0.32193683]),
              'j': array([   0, 1033,    2, ..., 1159, 1459, 2451]),
              'k': array([1144, 1125,    5, ..., 2502, 2502, 2502]),
              'type': 'mesh3d',
              'uid': '5e9d4f26-59aa-470f-aa67-017c9865a0fd',
              'x': array([-0.644662, -0.611354, -0.02151 , ..., -0.216801,  0.051458, -0.298077]),
              'y': array([ 1.049186,  0.742713,  0.884485, ..., -0.001668,  0.032704,  0.225157]),
              'z': array([ 0.056302,  0.03836 ,  0.135143, ...,  0.315951, -0.253654,  0.088463])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
        